# Prototyping

In [1]:
import pandas as pd
import json
import os

HOME=os.environ["HOME"]

In [2]:
fn = f"{HOME}/github/datalake/raw/aemet/predictions/hourly/20211216T122138/18153.json"

with open(fn, "r", encoding='latin-1') as f:
    j = json.load(f)

In [3]:
j

[{'origen': {'productor': 'Agencia Estatal de Meteorología - AEMET. Gobierno de España',
   'web': 'https://www.aemet.es',
   'enlace': 'https://www.aemet.es/es/eltiempo/prediccion/municipios/horas/peligros-id18153',
   'language': 'es',
   'copyright': '© AEMET. Autorizado el uso de la información y su reproducción citando a AEMET como autora de la misma.',
   'notaLegal': 'https://www.aemet.es/es/nota_legal'},
  'elaborado': '2021-12-16T10:42:53',
  'nombre': 'Peligros',
  'provincia': 'Granada',
  'prediccion': {'dia': [{'estadoCielo': [{'value': '11n',
       'periodo': '07',
       'descripcion': 'Despejado'},
      {'value': '11n', 'periodo': '08', 'descripcion': 'Despejado'},
      {'value': '11', 'periodo': '09', 'descripcion': 'Despejado'},
      {'value': '11', 'periodo': '10', 'descripcion': 'Despejado'},
      {'value': '11', 'periodo': '11', 'descripcion': 'Despejado'},
      {'value': '11', 'periodo': '12', 'descripcion': 'Despejado'},
      {'value': '11', 'periodo': '13

In [4]:
"""
\0]
|-"elaborado"
|-"nombre"
|-"provincia"
|-"prediccion"/"dia" 0,1,2]
|--0]"fecha"
|--0]"orto"
|--0]"ocaso"
|--0]"precipitacion", 'probPrecipitacion', 'probTormenta', 'nieve', 'probNieve', 'temperatura', 'sensTermica', 'humedadRelativa'
|---"value"
|---"periodo"
|--0]"estadoCielo"
|---"descripcion"
|---"value"
|---"periodo"
|--0]"vientoAndRachaMax"
|---"direccion"
|---"velocidad"
|---"value"
|---"periodo"

"""

'\n\\[0]\n|-"elaborado"\n|-"nombre"\n|-"provincia"\n|-"prediccion"/"dia" [0,1,2]\n|--[0]"estadoCielo"\n|---"value"\n|---"periodo"\n|---"descripcion"\n\n\'precipitacion\', \'probPrecipitacion\', \'probTormenta\', \'nieve\', \'probNieve\', \'temperatura\', \'sensTermica\', \'humedadRelativa\', \'vientoAndRachaMax\', \'fecha\', \'orto\', \'ocaso\'\n\n'

In [5]:
d = j[0]
res = None

In [10]:
print(d"elaborado"], d["nombre"], d["provincia"])
print(d["prediccion"]["dia"][0]["fecha"], d["prediccion"]["dia"][0]["orto"], d["prediccion"]["dia"][0]["ocaso"])

2021-12-16T10:42:53 Peligros Granada
2021-12-16T00:00:00 08:21 17:58


In [6]:
len(d["prediccion"]["dia"]) #.values()

3

In [7]:
d["prediccion"]["dia"][0].keys()

dict_keys(['estadoCielo', 'precipitacion', 'probPrecipitacion', 'probTormenta', 'nieve', 'probNieve', 'temperatura', 'sensTermica', 'humedadRelativa', 'vientoAndRachaMax', 'fecha', 'orto', 'ocaso'])

In [11]:
for k in ('estadoCielo', 'precipitacion', 'probPrecipitacion', 'probTormenta', 'nieve', 'probNieve', 'temperatura', 'sensTermica', 'humedadRelativa', 'vientoAndRachaMax'):
    print(k, str(d["prediccion"]["dia"][0][k])[:200])
    print("\n\n")

estadoCielo [{'value': '11n', 'periodo': '07', 'descripcion': 'Despejado'}, {'value': '11n', 'periodo': '08', 'descripcion': 'Despejado'}, {'value': '11', 'periodo': '09', 'descripcion': 'Despejado'}, {'value': '



precipitacion [{'value': '0', 'periodo': '07'}, {'value': '0', 'periodo': '08'}, {'value': '0', 'periodo': '09'}, {'value': '0', 'periodo': '10'}, {'value': '0', 'periodo': '11'}, {'value': '0', 'periodo': '12'}, {



probPrecipitacion [{'value': '0', 'periodo': '0713'}, {'value': '0', 'periodo': '1319'}, {'value': '0', 'periodo': '1901'}]



probTormenta [{'value': '0', 'periodo': '0713'}, {'value': '0', 'periodo': '1319'}, {'value': '0', 'periodo': '1901'}]



nieve [{'value': '0', 'periodo': '07'}, {'value': '0', 'periodo': '08'}, {'value': '0', 'periodo': '09'}, {'value': '0', 'periodo': '10'}, {'value': '0', 'periodo': '11'}, {'value': '0', 'periodo': '12'}, {



probNieve [{'value': '0', 'periodo': '0713'}, {'value': '0', 'periodo': '1319'}, {'value': '0', 'periodo': 

# Implementation

In [105]:
%reset -f

In [106]:
import pandas as pd
import json
import os

HOME = os.environ["HOME"]
INPUTDIR = "github/datalake/raw/aemet/predictions/hourly/20211216T122138"
fn = "{}/{}/18153.json".format(HOME, INPUTDIR)

"""
\root json file
-[0]
|-"elaborado"
|-"nombre"
|-"provincia"
|-"prediccion"/"dia" [0,1,2]
|--[X]"fecha"
|--[X]"orto"
|--[X]"ocaso"
|--[X]"precipitacion", 'probPrecipitacion', 'probTormenta', 'nieve', 'probNieve', 'temperatura', 'sensTermica', 'humedadRelativa'
|---"value"
|---"periodo"
|--[X]"estadoCielo"
|---"descripcion"
|---"value"
|---"periodo"
|--[X]"vientoAndRachaMax" (ciclos de 2 listas, indices 1-impar y 2-par)
|---[impar]"direccion"
|---[impar]"velocidad"
|---[impar]"periodo"
|---[par]"periodo"
|---[par]"value"

"""

def extractJsonHourlyAemetPredictions(f):
    """Extract info from json file for hourly aemet predictions."""
    j = None
    with open(f, "r", encoding='latin-1') as fo:
        j = json.load(fo)

    if j is None:
        return None
    
    # Create dataframe as main result variable
    res_main = pd.DataFrame([])
   
    # Extract json part for predictions
    d = j[0]["prediccion"]["dia"]  
    
    for n in range(len(d)):
        
        # Create dataframe as sub (loop) result variable
        res_sub = pd.DataFrame([], columns=["parametro", "value", "periodo"])
           
        # First group with similar structure.
        for k in ("precipitacion", 'probPrecipitacion', 'probTormenta', 'nieve', 'probNieve', 'temperatura', 'sensTermica', 'humedadRelativa'):   
            
            # Flatten structure and add to dataset.
            u = pd.json_normalize(d[n], k)
            u["parametro"] = k
            res_sub = pd.concat([res_sub, u])
            
        # Second "group" with similar structure.
        # Flatten structure and add to dataset.
        u = pd.json_normalize(d[n], "estadoCielo")
        u["parametro"] = "estadoCielo"
        res_sub = pd.concat([res_sub, u])

        # Third "group" with similar structure.
        u = pd.json_normalize(d[n]["vientoAndRachaMax"][::2])
        # the structure here has even and odd difference which is resolved by joining the two parts.
        u = u.merge(pd.json_normalize(d[n]["vientoAndRachaMax"][1::2]), on="periodo")
        u["parametro"] = "vientoAndRachaMax"
        res_sub = pd.concat([res_sub, u])

        # Add the common attributes of this part.
        res_sub["fecha"] = d[n]["fecha"]
        res_sub["orto"] = d[n]["orto"]
        res_sub["ocaso"] = d[n]["ocaso"]
        
        # Finally concat this data to the main df.
        res_main = pd.concat([res_main, res_sub])
    
    # Main loop: add the common attributes of this part.
    res_main["elaborado"] = j[0]["elaborado"]
    res_main["nombre"] = j[0]["nombre"]
    res_main["provincia"] = j[0]["provincia"]
    
    return res_main


# Test

In [107]:
json_flattened = extractJsonHourlyAemetPredictions(fn)
json_flattened

,parametro,value,periodo,descripcion,direccion,velocidad,fecha,orto,ocaso,elaborado,nombre,provincia
0,precipitacion,0,07,NaN,NaN,NaN,2021-12-16T00:00:00,08:21,17:58,2021-12-16T10:42:53,Peligros,Granada
1,precipitacion,0,08,NaN,NaN,NaN,2021-12-16T00:00:00,08:21,17:58,2021-12-16T10:42:53,Peligros,Granada
2,precipitacion,0,09,NaN,NaN,NaN,2021-12-16T00:00:00,08:21,17:58,2021-12-16T10:42:53,Peligros,Granada
3,precipitacion,0,10,NaN,NaN,NaN,2021-12-16T00:00:00,08:21,17:58,2021-12-16T10:42:53,Peligros,Granada
4,precipitacion,0,11,NaN,NaN,NaN,2021-12-16T00:00:00,08:21,17:58,2021-12-16T10:42:53,Peligros,Granada
...,...,...,...,...,...,...,...,...,...,...,...,...
2,vientoAndRachaMax,6,02,NaN,[NE],[5],2021-12-18T00:00:00,08:22,17:59,2021-12-16T10:42:53,Peligros,Granada
3,vientoAndRachaMax,7,03,NaN,[NE],[5],2021-12-18T00:00:00,08:22,17:59,2021-12-16T10:42:53,Peligros,Granada
4,vientoAndRachaMax,6,04,NaN,[NE],[5],2021-12-18T00:00:00,08:22,17:59,2021-12-16T10:42:53,Peligros,Granada
5,vientoAndRachaMax,4,05,NaN,[E],[4],2021-12-18T00:00:00,08:22,17:59,2021-12-16T10:42:53,Peligros,Granada


In [ ]:
# Create table with NO normalizaton.
CREATE TABLE prediccion_horaria_nonorm (
    elaborado TIMESTAMP NOT NULL,
    nombre VARCHAR(50) NOT NULL,
    provincia VARCHAR(50) NOT NULL,
    fecha TIMESTAMP NOT NULL,
    orto CHAR(5) NOT NULL,
    ocaso CHAR(5) NOT NULL,
    parametro VARCHAR(50) NOT NULL,
    value NUMERIC(4,2) NOT NULL,
    periodo INTEGER NOT NULL,
    descripcion VARCHAR(100),
    direccion VARCHAR(5),
    velocidad VARCHAR(5)
);

In [ ]:
# Insert one line to table.
INSERT INTO prediccion_horaria_nonorm(parametro, value, periodo, descripcion, direccion, velocidad, fecha, orto, ocaso, elaborado, nombre, provincia)
VALUES ('precipitacion', 0, 07, 'NaN', 'NaN', 'NaN', '2021-12-16T00:00:00', '08:21', '17:58', '2021-12-16T10:42:53', 'Peligros', 'Granada');

In [ ]:
# Print the whole table.
SELECT * FROM prediccion_horaria_nonorm;

In [ ]:
# bulk insert.
INSERT INTO prediccion_horaria_nonorm(parametro, value, periodo, descripcion, direccion, velocidad, fecha, orto, ocaso, elaborado, nombre, provincia)
VALUES
('precipitacion', '0', '08', 'NaN', 'NaN', 'NaN', '2021-12-16T00:00:00', '08:21', '17:58', '2021-12-16T10:42:53', 'Peligros', 'Granada'),
('precipitacion', '0', '09', 'NaN', 'NaN', 'NaN', '2021-12-16T00:00:00', '08:21', '17:58', '2021-12-16T10:42:53', 'Peligros', 'Granada'),
('precipitacion', '0', '10', 'NaN', 'NaN', 'NaN', '2021-12-16T00:00:00', '08:21', '17:58', '2021-12-16T10:42:53', 'Peligros', 'Granada'),
('precipitacion', '0', '11', 'NaN', 'NaN', 'NaN', '2021-12-16T00:00:00', '08:21', '17:58', '2021-12-16T10:42:53', 'Peligros', 'Granada'),
('vientoAndRachaMax', '6', '02', 'NaN', 'NE', '5', '2021-12-18T00:00:00', '08:22', '17:59', '2021-12-16T10:42:53', 'Peligros', 'Granada'),
('vientoAndRachaMax', '7', '03', 'NaN', 'NE', '5', '2021-12-18T00:00:00', '08:22', '17:59', '2021-12-16T10:42:53', 'Peligros', 'Granada'),
('vientoAndRachaMax', '6', '04', 'NaN', 'NE', '5', '2021-12-18T00:00:00', '08:22', '17:59', '2021-12-16T10:42:53', 'Peligros', 'Granada'),
('vientoAndRachaMax', '4', '05', 'NaN', 'E', '4', '2021-12-18T00:00:00', '08:22', '17:59', '2021-12-16T10:42:53', 'Peligros', 'Granada'),
('vientoAndRachaMax', '4', '06', 'NaN', 'SE', '3', '2021-12-18T00:00:00', '08:22', '17:59', '2021-12-16T10:42:53', 'Peligros', 'Granada');


In [ ]:
# include id as type SERIAL
ALTER TABLE prediccion_horaria_nonorm
ADD COLUMN id SERIAL;